### Pendant Drop Tensiometry Regression Models
#### Algorithms
* XGBoost
* LightGBM
#### Data:
* This model takes the labeled set of features of the pendant drop profile and becomes a function of beta.
* Input features include Drop Height, Capillary Radius, R-s, R-e, and Smax.


The current model is trained, tested, and tuned on dataset (data/pdt-dataset.csv) which has 2500 entries.


In [1]:
import pandas as pd
import pickle
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, StratifiedKFold

from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import lightgbm as lgb

In [20]:
# I am multiplying all elements by 10^6, to keep float integrity when using gridsearchCV as int64
df = pd.read_csv('../data/pdt-dataset.csv').apply(lambda x: x*1000000).astype('int64')
df.head()

,Drop Height,Capillary Radius,R-s,R-e,Smax,Beta
0,2931984,675340,890067,1085611,3590000,400000
1,3041870,673539,885918,1094985,3689763,400000
2,3131424,666058,889661,1085394,3789526,400000
3,3235418,674748,884866,1085809,3889289,400000
4,3335846,689308,896088,1086557,3989053,400000


In [21]:
# This model predicts Beta given a Pendant Drop Profile
X = df.drop('Beta', axis=1)
y = df['Beta']

# Stratified fold includes the same percentage of target values in each fold.
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)
y.head()

0    400000
1    400000
2    400000
3    400000
4    400000
Name: Beta, dtype: int64

In [22]:
# This function takes a list of hyperparameter configs and finds the best one.
def grid_search(params, random=False):
    # Initialize XGB Regressor with objective='reg:squarederror' (MSE)
    xgb = XGBRegressor(booster='gbtree', objective='reg:squarederror',
    random_state=2)
    if random:
        grid = RandomizedSearchCV(xgb, params, cv=kfold, n_iter=20, n_jobs=-1)
    else:
        grid = GridSearchCV(xgb, params, cv=kfold, n_jobs=-1)
    grid.fit(X, y)
    best_params = grid.best_params_
    print("Best params:", best_params)
    best_score = grid.best_score_
    print("Training score: {:.3f}".format(best_score))

In [23]:
grid_search(params={'n_estimators': [100, 200, 400, 800]})

Best params: {'n_estimators': 800}
Training score: 0.999


In [24]:
grid_search(params={'learning_rate':[0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]})

Best params: {'learning_rate': 0.1}
Training score: 0.999


In [25]:
grid_search(params={'max_depth':[2, 3, 5, 6, 8]})

Best params: {'max_depth': 8}
Training score: 0.999


### Tuned XGBoost Regressor
* n-estimators: 800
* learning_rate=.1
* max_depth = 5

Accuracy score on test data (.999), RMSE: (0.0034324513493428823)



In [2]:
# Build, train, test, and save our model
xgb = XGBRegressor(booster='gbtree', objective='reg:squarederror',
    random_state=2, learning_rate=.1, n_estimators=800, max_depth=5)

df = pd.read_csv('../data/pdt-dataset.csv')
X = df.drop('Beta', axis=1)
y = df['Beta']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

xgb.fit(X_train, y_train)

y_pred = xgb.predict(X_test)

reg_mse = mean_squared_error(y_test, y_pred)
reg_rmse = np.sqrt(reg_mse)

print(f"RMSE: {reg_rmse}")
# print(lgbm_reg.feature_importances_)
norm_mean = reg_rmse / np.mean(y)
print(f"Normalized Mean {norm_mean}")

RMSE: 0.003500566413769064
Normalized Mean 0.005834277356281772


In [ ]:
# Save our previous model to models folder
with open("../models/pdt-regression-model.pkl", 'wb') as f:
    pickle.dump(xgb, f)

An example of how to use saved models.

In [ ]:
# Load the model from models folder
with open("../models/pdt-regression-model.pkl", 'rb') as f:
    model = pickle.load(f)

#### Experimenting with wider beta range data set (.1-.8) on same model

In [3]:
# Build, train, test, and save our model
xgb = XGBRegressor(booster='gbtree', objective='reg:squarederror',
    random_state=2, learning_rate=.1, n_estimators=800, max_depth=5)

df = pd.read_csv('../data/pdt-dataset-wider-beta.csv')
X = df.drop('Beta', axis=1)
y = df['Beta']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

xgb.fit(X_train, y_train)

y_pred = xgb.predict(X_test)

reg_mse = mean_squared_error(y_test, y_pred)
reg_rmse = np.sqrt(reg_mse)
print(reg_rmse)


# let's test on original data
df = pd.read_csv('../data/pdt-dataset.csv')
X = df.drop('Beta', axis=1)
y = df['Beta']
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.3, random_state=42)

y_pred = xgb.predict(X_test)

reg_mse = mean_squared_error(y_test, y_pred)
reg_rmse = np.sqrt(reg_mse)
print(f"RMSE: {reg_rmse}")
# print(lgbm_reg.feature_importances_)
norm_mean = reg_rmse / np.mean(y)
print(f"Normalized Mean {norm_mean}")

0.004374367554479346
RMSE: 0.00474501474542218
Normalized Mean 0.007908357909036965


#### Experiment with same model but without Smax as training data and larger range of beta

In [4]:
# Build, train, test, and save our model
xgb = XGBRegressor(booster='gbtree', objective='reg:squarederror',
    random_state=2, learning_rate=.1, n_estimators=800, max_depth=5)

df = pd.read_csv('../data/pdt-dataset-wider-beta-no-Smax.csv')
X = df.drop('Beta', axis=1)
y = df['Beta']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

xgb.fit(X_train, y_train)

y_pred = xgb.predict(X_test)

reg_mse = mean_squared_error(y_test, y_pred)
reg_rmse = np.sqrt(reg_mse)
print(f"RMSE: {reg_rmse}")
# print(lgbm_reg.feature_importances_)
norm_mean = reg_rmse / np.mean(y)
print(f"Normalized Mean {norm_mean}")

RMSE: 0.004334942946277289
Normalized Mean 0.009633206547282863


### Build and Characterize a LightGBM Regressor Model for the same 3 datasets.

In [3]:
df = pd.read_csv('../data/pdt-dataset-wider-beta-no-Smax.csv').apply(lambda x: x*1000000).astype('int64')
df.head()

,Drop Height,Capillary Radius,R-s,R-e,Beta
0,2305560,98640,469920,1017619,100000
1,2322946,83864,460409,1017719,100000
2,2347761,78837,440249,1017397,100000
3,2372782,81146,454684,1019378,100000
4,2398734,91546,461319,1017129,100000


In [4]:
# This model predicts Beta given a Pendant Drop Profile
X = df.drop('Beta', axis=1)
y = df['Beta']

# Stratified fold includes the same percentage of target values in each fold.
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)
y.head()

0    100000
1    100000
2    100000
3    100000
4    100000
Name: Beta, dtype: int64

In [5]:
# This function takes a list of hyperparameter configs and finds the best one.
def lgb_grid_search(params, random=False):
    # Initialize LightGBM Regressor with objective='reg:squarederror' (MSE)
    lgb_reg = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression',
    random_state=2)
    if random:
        grid = RandomizedSearchCV(lgb_reg, params, cv=kfold, n_iter=20, n_jobs=-1)
    else:
        grid = GridSearchCV(lgb_reg, params, cv=kfold, n_jobs=-1)
    grid.fit(X, y)
    best_params = grid.best_params_
    print("Best params:", best_params)
    best_score = grid.best_score_
    print("Training score: {:.3f}".format(best_score))

In [6]:
param_list = [
    {'learning_rate': [0.1, 0.01], 'max_depth': [3, 5]},
    {'learning_rate': [0.05, 0.01], 'max_depth': [4, 6]}
]
best_regressor = lgb_grid_search(param_list, False)

Best params: {'learning_rate': 0.1, 'max_depth': 5}
Training score: 1.000


In [7]:
param_list = {
    'num_leaves': [10, 20, 50],  # Maximum number of leaves in a tree
    'max_depth': [5, 10, 20],  # Maximum depth of a tree
    'learning_rate': [0.01, 0.05, 0.1],  # Learning rate for gradient boosting
    'n_estimators': [50, 100, 250],  # Number of trees to fit
    # 'subsample_for_bin': [20000, 50000, 100000],  # Number of samples for constructing bins
    'min_child_samples': [5, 10, 20],  # Minimum number of samples required to form a leaf node
    #'reg_alpha': [0, 0.1, 0.5, 1],  # L1 regularization
    #'reg_lambda': [0, 0.1, 0.5, 1],  # L2 regularization
    #'colsample_bytree': [0.5, 0.7, 1.0],  # Fraction of features to consider at each split
    #'subsample': [0.5, 0.7, 1.0],  # Fraction of samples to use for each tree
    #'min_split_gain': [0, 0.1, 0.5],  # Minimum loss reduction required to form a new split
    #'max_bin': [255, 511, 1023],  # Maximum number of bins to use
    'early_stopping_rounds': [None, 5, 10, 15],  # Early stopping based on the validation set score
}
best_regressor = lgb_grid_search(param_list, False)

Best params: {'early_stopping_rounds': None, 'learning_rate': 0.1, 'max_depth': 20, 'min_child_samples': 10, 'n_estimators': 250, 'num_leaves': 50}
Training score: 1.000


### Tuned LightGBM Model
* early_stopping_rounds=None
* learnging_rate=0.1
* max_depth=20
* min_child_samples=10
* n_estimators=250
* num_leaves=50

In [5]:
# Build, train, test, and save our LightGBM model
lgbm_reg = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression',
                            early_stopping_rounds=None,
                            learning_rate=0.1,
                            max_depth=20,
                            min_child_samples=10,
                            n_estimators=250,
                            num_leaves=50,
                            random_state=2)

df = pd.read_csv('../data/pdt-dataset-wider-beta-no-Smax.csv')

X = df.drop('Beta', axis=1)
y = df['Beta']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

lgbm_reg.fit(X_train, y_train)

y_pred = lgbm_reg.predict(X_test)

reg_mse = mean_squared_error(y_test, y_pred)
reg_rmse = np.sqrt(reg_mse)
print(f"RMSE: {reg_rmse}")
# print(lgbm_reg.feature_importances_)
norm_mean = reg_rmse / np.mean(y)
print(f"Normalized Mean {norm_mean}")

RMSE: 0.004259963266041081
Normalized Mean 0.009466585035646846


In [ ]:
# Define the custom scoring function that calculates normalized MSE
def normalized_mse(y_true, y_pred):
    mse = np.mean((y_true - y_pred)**2)
    norm_factor = np.mean(y_true)
    normalized_mse = mse / norm_factor**2
    return -normalized_mse   # return negative value to minimize the metric

# We can now start a grid search with our custom scoring function
grid_search = GridSearchCV(estimator=xgb_reg, param_grid=param_grid, scoring=make_scorer(normalized_mse, greater_is_better=False), cv=5)